In [2]:
import json
import random
from datetime import datetime, timedelta
from time import sleep
import pandas as pd
from kafka import KafkaProducer

KAFKA_TOPIC = 'streaming'
SERVER = "broker:9092"
LAG = 10

def bike_random(row):
    bike_number = random.randint(0, row['Number of slots'])
    e_bike_number = random.randint(0, row['Number of slots'] - bike_number)
    empty_slots = row['Number of slots'] - bike_number - e_bike_number
    return bike_number, e_bike_number, empty_slots

def generate_json(df):
    all_messages = []
    current_time = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")
    for index, row in df.iterrows():
        bike_number, e_bike_number, empty_slots = bike_random(row)
        message = {
            "station_name": row['Station Name'],
            "latitude": row['Latitude'],
            "longitude": row['Longitude'],
            "number_of_slots": row['Number of slots'],
            "number_of_free_bikes": bike_number,
            "number_of_e_bikes": e_bike_number,
            "number_of_empty_slots": empty_slots,
            "update_time": current_time
        }
        all_messages.append(message)
    return all_messages

if __name__ == "__main__":
    df = pd.read_excel("bike.xlsx")
    producer = KafkaProducer(
        bootstrap_servers=[SERVER],
        value_serializer=lambda x: json.dumps(x).encode("utf-8"),
        api_version=(3, 7, 0),
    )
    
    try:
        while True:
            all_messages = generate_json(df)
            producer.send(KAFKA_TOPIC, value=all_messages)
            sleep(LAG)

    except KeyboardInterrupt:
        producer.close()


KafkaTimeoutError: KafkaTimeoutError: Failed to update metadata after 60.0 secs.